In [ ]:
!pip install Pillow
!pip install matplotlib
!pip install opencv-python

In [12]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [13]:
def interpolate(value, start, end):
    """Interpolate a value between a start and end point."""
    return start + (end - start) * value

'''
min_value should be 0k and max_value should be 400k for temp
min_value should be 0 and max_value should be 45 for slope
'''
def color_to_value(color, min_value, max_value, scale):
    #slope_min = rgb(67,0,71)
    #slope_max = rgb(255,255,255)
    #temp_min = rgb(244,81,255)
    #temp_max = rgb(255,255,255)

    # Define variables
    norm_min_color = np.array([0,0,0]) / 255.0
    norm_max_color = np.array([255,255,255]) / 255.0

    if scale == 'slope':
        norm_min_color = np.array([67,0,71]) / 255.0
        norm_max_color = np.array([255,255,255]) / 255.0
    if scale == 'temp':
        norm_min_color = np.array([244,81,255]) / 255.0
        norm_max_color = np.array([255,255,255]) / 255.0
    if scale == 'tio2':
        norm_min_color = np.array([68,68,68]) / 255.0
        norm_max_color = np.array([255,255,255]) / 255.0

    norm_color = np.array(color) / 255.0
    
    # Calculate the relative position of the color in the gradient
    # Avoid division by zero for a color that matches exactly the min_color
    denom = (norm_max_color - norm_min_color)
    denom = np.where(denom == 0, 1, denom)  # Protect against division by zero
    relative_position = (norm_color - norm_min_color) / denom
    
    # Use the position to interpolate between the min and max values
    # If the color is exactly min_color, set position to 0 (start of scale)
    relative_position = np.where(denom == 1, 0, relative_position)
    
    # Assuming the color gradient is linear and the scale is uniform
    value = np.mean(relative_position)
    
    # Interpolate the numerical value
    numerical_value = interpolate(value, min_value, max_value)
    return numerical_value

In [27]:
image1 = cv2.imread('./slope.png')
image2 = cv2.imread('./heat.png')
image3 = cv2.imread('./tio2.png')
# Assuming the image is already in the color space you're mapping from
height, width, channels = image1.shape

# Initialize an array for your scalar values
slope_map = np.zeros((height, width))
tio2_map = np.zeros((height, width))
heat_map = np.zeros((height, width))

for i in range(height):
    for j in range(width):
        color = image1[i, j]
        color2 = image2[i, j]
        color3 = image3[i, j]

        slope = color_to_value(color, 1, 45, 'slope')
        heat = color_to_value(color2, 1, 400, 'heat')
        tio2 = color_to_value(color3, 0.1, 0.11, 'tio2')

        slope_map[i, j] = slope
        heat_map[i, j] = heat
        tio2_map[i, j] = tio2

In [28]:
def estimate_value_of_profit(x,y):
    slope_val = slope_map[x,y]
    heat_val = heat_map[x,y]
    tio2_val = tio2_map[x,y]
    he_val = 1

    if tio2_val > 0.05:
        he_val = 15
    if tio2_val > 0.03 and tio2_val < 0.05:
        he_val = 8
    if tio2_val > 0.01 and tio2_val < 0.03:
        he_val = 3
    if tio2_val < 0.01:
        he_val = 1
        
    print(he_val)
    print(tio2_val)
    
    return (5 * 0.0047**2 * 1900 * tio2_val * (32000*40000) / (he_val * 0.1 * slope_val * heat_val))

In [29]:
estimate_value_of_profit(100, 243)

15
0.11


2562501.3180368645